# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
weather_data_csv = 'output_data/weather_df.csv'
weather_data_df = pd.read_csv(weather_data_csv)
weather_data_df

,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Longitude,Latitude
0,mponela,16.65,95,21,1.72,MW,1643669418,33.7401,-13.5319
1,srivardhan,20.04,56,10,2.63,IN,1643669419,73.0167,18.0333
2,kytmanovo,-12.90,97,100,1.26,RU,1643669419,85.4479,53.4593
3,hofn,-0.48,88,96,3.80,IS,1643669420,-15.2082,64.2539
4,butaritari,27.39,79,93,5.82,KI,1643669421,172.7902,3.0707
...,...,...,...,...,...,...,...,...,...
582,tagusao,25.56,85,11,2.14,PH,1643669874,117.8141,9.1924
583,grimshaw,-17.22,55,40,5.66,CA,1643669874,-117.6027,56.1834
584,verkhnevilyuysk,-34.62,100,96,1.45,RU,1643669875,120.3167,63.4453
585,concepcion del oro,16.26,14,0,8.17,MX,1643669876,-101.4167,24.6333


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [11]:
locations = weather_data_df[['Latitude', 'Longitude']]
humidity = weather_data_df['Humidity']
worldmap = gmaps.figure(center = [0,0], zoom_level=2)
heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius=2)
worldmap.add_layer(heatmap)
worldmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

ideal_weather = weather_data_df.loc[
  # A max temperature lower than 80 degrees but higher than 70.
    (weather_data_df['Temperature'] < 89) & (weather_data_df['Temperature'])
  # Wind speed less than 10 mph.
    & (weather_data_df['Wind Speed'] < 10)
  # Zero cloudiness.
    & (weather_data_df['Cloudiness'] == 0)]

ideal_weather
    
 # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.



,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Longitude,Latitude
11,kapaa,24.08,71,0,1.34,US,1643669426,-159.3190,22.0752
13,dhule,18.97,35,0,2.64,IN,1643669428,74.7833,20.9000
14,jamestown,-2.00,50,0,2.06,US,1643669142,-79.2353,42.0970
32,lebu,15.22,72,0,8.04,CL,1643669290,-73.6500,-37.6167
37,benjamin aceval,36.08,16,0,2.71,PY,1643669447,-57.5667,-24.9667
...,...,...,...,...,...,...,...,...,...
546,praya,21.30,88,0,5.66,CV,1643669845,-23.5087,14.9215
571,arlit,13.14,20,0,2.26,NE,1643669866,7.3853,18.7369
574,dourbali,18.04,14,0,6.06,TD,1643669868,15.8632,11.8091
579,saubara,27.98,80,0,3.85,BR,1643669871,-38.7686,-12.7375


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = ideal_weather[['City', 'Country', 'Latitude', 'Longitude']]
hotel_df['Hotel'] = ''
hotel_df

/var/folders/mz/2rxvtwjx30zc8fr83rlnl3zh0000gn/T/ipykernel_60601/3016362562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel'] = ''


,City,Country,Latitude,Longitude,Hotel
11,kapaa,US,22.0752,-159.3190,
13,dhule,IN,20.9000,74.7833,
14,jamestown,US,42.0970,-79.2353,
32,lebu,CL,-37.6167,-73.6500,
37,benjamin aceval,PY,-24.9667,-57.5667,
...,...,...,...,...,...
546,praya,CV,14.9215,-23.5087,
571,arlit,NE,18.7369,7.3853,
574,dourbali,TD,11.8091,15.8632,
579,saubara,BR,-12.7375,-38.7686,


In [23]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    location = f'{latitude},{longitude}'
    params = {'location':location,'radius':5000,'types':'lodging','key':g_key}
    response = requests.get(base_url, params=params)
    response = response.json()
    try:
        hotel_df.loc[index, "Hotel"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print('Missing field result skipped.')

hotel_df

/Users/joshuabuckwalter/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field result skipped.
Missing field result skipped.
Missing field result skipped.


,City,Country,Latitude,Longitude,Hotel
11,kapaa,US,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,dhule,IN,20.9000,74.7833,Hotel Dwarkamai
14,jamestown,US,42.0970,-79.2353,Hampton Inn & Suites Jamestown
32,lebu,CL,-37.6167,-73.6500,Hostal Las Lilas
37,benjamin aceval,PY,-24.9667,-57.5667,Escuela Agrícola y Hotel Cerrito
...,...,...,...,...,...
546,praya,CV,14.9215,-23.5087,LT Aparthotel
571,arlit,NE,18.7369,7.3853,Case de Passage De la Prefecture
574,dourbali,TD,11.8091,15.8632,
579,saubara,BR,-12.7375,-38.7686,Enseada do Caeiro Eco Resort


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel location: {rate}" for rate in hotel_info])

worldmap.add_layer(markers)
# Display figure
worldmap


Figure(layout=FigureLayout(height='420px'))